## IMPORTS-EXPORTS FROM THE EU: TIME SERIES, VISUALITAZION AND CLUSTERING

In this exercise I am going to use diverse methods in order to understand the trading behaviours of the countries beside the EU (conformed by the 28 states) resulting for that a different categories in order of importance

### Practical objective of the exercise:
    - Better comprehension of forloops and if clauses
    - Easy aplication of machine learning (clustering Kmeans)
    - Matplot views
    - Reporting: Tableau

Firstly, we import the necesary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
os.path # our path

### Importing our data

Source: Eurostat

The format of our file is excel. 


In [ ]:
df = pd.read_excel("eu28_allyears.xlsx",skiprows=4,skipfooter=7)
df.tail()

### Cleaning and ordering data

 1) Cleaning unnecesary columns
 
 2) Renaming the columns for better understanding
 
 3) Dropping unnecesary information and reindexing

In [ ]:
# Drop unnecesary columns

name_cols = []

for i in df.columns:
    if i.startswith("Import Supl Unit") or i.startswith("Export Supl Unit"):
        pass
    else:
        name_cols.append(i)

In [ ]:
df = df[name_cols]

In [ ]:
# Renaming

name_cols = []

for position, i in enumerate(df.columns):
    
    if position==0:
        name_cols.append(i)
    elif i.startswith("Import Value to"):
        word = "import" + "_" + df.iloc[1,position]
        name_cols.append(word)
    elif i.startswith("Import Qty to the"):
        word = "import_Qty" + "_" + df.iloc[1,position]
        name_cols.append(word)         
    elif i.startswith("Export Value from"):
        word = "export" + "_" + df.iloc[1,position]
        name_cols.append(word)      
    elif i.startswith("Export Qty from the"):
        word = "export_Qty" + "_" + df.iloc[1,position]
        name_cols.append(word)    
    else:
        pass
    


In [ ]:
dict_name = dict(zip(df.columns,name_cols))
dict_name

In [ ]:
df.rename(columns=dict_name,inplace=True)

In [ ]:
# Dropping information adn reset
df = df.drop([0,1,2],axis=0)
df.reset_index(drop=True,inplace=True)
df.head()

In [ ]:
df.set_index(keys="Indicators",inplace=True)

In [ ]:
df.info() # to understand our dataframe

In [ ]:
df.head(3)

### Visualization

I put it under # symbol to not be ejecuted

In [ ]:
# visualization of imports to EU
#for country in df.index:
   # plt.figure(figsize=(20,5))
   #  plt.plot(df.columns[0:16].values,df.loc[country][0:16].values)
   # plt.xlabel(df.loc[country][0:1].values)
   # plt.ylabel("euros")
   # plt.show()

In [ ]:
# visualization of exports from EU
#for country in df.index:
   # plt.figure(figsize=(20,5))
   # plt.plot(df.columns[32:48].values,df.loc[country][32:48].values)
   # plt.xlabel(df.loc[country][0:1].values)
   # plt.ylabel("euros")
   # plt.show()

# CLUSTERING EXPORTS-IMPORTS

In this section I am going to import and use the Kmeans algorithm from the sklearn library that will separate for us the diverse countries in 3 groups, attending to the euclidian distance of his position (settled by the exports and the imports in the last year of the dataframe)

In [ ]:
X = df[["import_2018","export_2018"]]
X.head(5)

In [ ]:
from sklearn.cluster import k_means
from sklearn.preprocessing import MinMaxScaler

##### Normalizing data.

As with any Machine Learning algorithm that uses distance functions, the data must be normalized before applying the algorithm. In this case we will use the MinMaxScaler () function that normalizes all data between [0, 1]

In [ ]:
min_max_scaler = MinMaxScaler() 
X_normalize = min_max_scaler.fit_transform(X)
X_normalize = pd.DataFrame(X_normalize) # Hay que convertir a DF el resultado.
X_normalize.rename(columns={0:"import_18",1:"export_18"},inplace=True)
X_normalize.set_index(X.index,inplace=True)

In [ ]:
X_normalize.head()

In [ ]:
# Graphic representation in order to see posible errors

x = X_normalize['import_18'].values
y = X_normalize['export_18'].values
plt.xlabel('importaciones')
plt.ylabel('exportaciones')
plt.title('import-export')
plt.plot(x,y,'o',markersize=3)

In [ ]:
k = 3
cluster = k_means(X,k)
cluster

In [ ]:
centroids = cluster[0]
centroids

In this section we incorporate the "importance" column to the dataframe

In [ ]:
df['importance'] = cluster[1]

In [ ]:
len(cluster[1])

In [ ]:
# Graphic representation with the clusters by color
plt.figure()
plt.scatter(X_normalize['import_18'].values,X_normalize['export_18'].values,c = df['importance'])
plt.xlabel('imports')
plt.ylabel("exports")
plt.show()

In [ ]:
df['importance'].values

The next step is to reasing the posible exit of the cluster number assignament to the data criteria. We will always have many similar countries and not much of them that stand out

In [ ]:
value_count = df['importance'].value_counts().sort_values(ascending=False)
#print(value_count)
#value_count.sort_values(ascending=False)
value_count

In [ ]:
name_dict = {}
for i in range(len(value_count.index)):
    if i==0:
        name_dict[value_count.index[i]] = "low"
    elif i==1:
        name_dict[value_count.index[i]] = "high"
    else:
        name_dict[value_count.index[i]] = "very_high"
        
name_dict
    

In [ ]:
df["importance"].replace(name_dict,inplace=True)

In [ ]:
df["importance"] #exit of the importance of the countries

In [ ]:
df_relevant = df [ df['importance'].isin(['very_high','high'])]
df_relevant

### Time series of the most relevant countries by the exports from the EU

In [ ]:
for country in df_relevant.index:
    plt.figure(figsize=(20,5))
    plt.plot(df_relevant.columns[32:48].values,df_relevant.loc[country][32:48].values)
    plt.xlabel(df_relevant.loc[country][0:1].values)
    plt.xlabel(country)
    plt.ylabel("euros")
    plt.show()

In [ ]:
df_relevant.sort_values(['import_2018'], ascending=False)

# Dash

In [ ]:
# pip install dash==1.6.0  # The core dash backend
# pip install dash-daq==0.2.1  # DAQ components (newly open-sourced!)

In [ ]:
#import dash_core_components
#print(dash_core_components.__version__)

In [ ]:
# En desarrollo

In [ ]:
# Already executed for Tableau
# import_exports = df.to_excel('import_exports.xls',sheet_name='df')

## TABLEAU

The reporting of this work will be in Tableau. 